In [1]:
import geopandas as gpd
import pandas as pd
import sys
import os
import rasterio
from rasterio.mask import mask
import numpy as np
import random

In [2]:
# Get the current working directory
current_dir = os.path.abspath('')

# Search for the 'constants.py' file starting from the current directory and moving up the hierarchy
project_root = current_dir
while not os.path.isfile(os.path.join(project_root, 'constants.py')):
    project_root = os.path.dirname(project_root)

# Add the project root to the Python path
sys.path.append(project_root)

In [3]:
from constants import DATA_PATH

In [4]:
dissolved_putid = r"C:\Users\bsf31\Documents\post-meds\data\policy-data\processing\clean_dissolved_clean_putid.gpkg"

In [5]:
# Load the dataset
gdf = gpd.read_file(dissolved_putid)

In [6]:
""" gdf.geometry = gdf.buffer(-60, join_style= 2)
gdf.geometry = gdf.buffer(60, join_style= 2)
 """

In [ ]:
gdf.crs

In [ ]:
# For Visual Check in Qgis

'''
output_path = os.path.join(DATA_PATH,'processing')


# Create the directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
    # Save the GeoDataFrame as a GeoPackage
# Define the filename for the GeoPackage

filename = os.path.join(output_path, "clean_dissolved_clean_putid.gpkg")
gdf.to_file(filename, driver="GPKG")'''

In [13]:
raster = r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\binary_deforestation_raster.tif"

In [20]:
""" 
def select_validation_set(gdf, raster_path, distance_threshold=5000, validation_fraction=0.1, seed=None):
    # Set the seed for reproducibility
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    with rasterio.open(raster_path) as src:
        raster_crs = src.crs
        raster_data = src.read(1)
        total_pixels = (raster_data != src.nodata).sum()
        print(f"Total non-NA pixels in raster: {total_pixels}")

    if gdf.crs != 'EPSG:32721':
        raise ValueError("The GeoDataFrame should be in EPSG:32721.")

    validation_set = gpd.GeoDataFrame(columns=gdf.columns)
    remaining_set = gdf.copy()

    target_pixel_count = validation_fraction * total_pixels
    accumulated_pixel_count = 0
    close_to_target_threshold = target_pixel_count * 0.99

    print(f"Target pixel count for validation: {target_pixel_count}, Seed: {seed}")

    while accumulated_pixel_count < target_pixel_count:
        if accumulated_pixel_count >= close_to_target_threshold:
            selected_property = remaining_set.sample(1, random_state=seed)
            print("Selecting final property to meet threshold...")
        else:
            selected_property = remaining_set.sample(1, random_state=seed)
            remaining_set = remaining_set.drop(selected_property.index)

            distances = remaining_set.distance(selected_property.geometry.squeeze())
            remaining_set = remaining_set.loc[distances > distance_threshold]

        selected_property_reprojected = selected_property.to_crs(raster_crs)
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, selected_property_reprojected.geometry, crop=True)
            selected_pixels = (out_image != src.nodata).sum()

        accumulated_pixel_count += selected_pixels
        validation_set = pd.concat([validation_set, selected_property], ignore_index=True)

        print(f"Selected ID: {selected_property.index[0]}, pixels: {selected_pixels}, accumulated: {accumulated_pixel_count}")

        if remaining_set.empty:
            print("No more properties available for selection.")
            break


    # Calculate the current percentage of the total raster area covered
    covered_percentage = (accumulated_pixel_count / total_pixels) * 100
    print(f"Covered Percentage: {covered_percentage}%")
    
    return validation_set """


In [ ]:
""" validation_set = select_validation_set(gdf, raster, seed = 1)
 """

In [25]:
""" 
# Save validation set to a GeoPackage file
validation_set.to_file("validation_set.gpkg", driver="GPKG")     """

# Split Vector to training, testing, validation

In [59]:


""" def select_vector_sets(gdf, raster_path, train_fraction=0.15, validation_fraction=0.10, distance_threshold=5000, seed=None):
    # Set the seed for reproducibility
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    with rasterio.open(raster_path) as src:
        raster_crs = src.crs
        raster_data = src.read(1)
        total_pixels = (raster_data != src.nodata).sum()
        print(f"Total non-NA pixels in raster: {total_pixels}")

    if gdf.crs != 'EPSG:32721':
        raise ValueError("The GeoDataFrame should be in EPSG:32721.")

    # Initialize GeoDataFrames for train, validation, and test sets
    train_set = gpd.GeoDataFrame(columns=gdf.columns)
    validation_set = gpd.GeoDataFrame(columns=gdf.columns)
    test_set = gdf.copy()

    # Calculate the target pixel counts for each set
    target_train_pixels = train_fraction * total_pixels
    target_validation_pixels = validation_fraction * total_pixels

    accumulated_train_pixels = 0
    accumulated_validation_pixels = 0

    close_to_train_threshold = target_train_pixels * 0.99
    close_to_validation_threshold = target_validation_pixels * 0.99

    print(f"Target pixel count for train: {target_train_pixels}, validation: {target_validation_pixels}, Seed: {seed}")

    while accumulated_train_pixels < target_train_pixels:
        if test_set.empty:
            print("No more properties available for train set selection.")
            break
        
        if accumulated_train_pixels >= close_to_train_threshold:
            print("Close to the train pixel threshold; selecting final properties...")
            selected_property = test_set.sample(1, random_state=seed)
        else:
            selected_property = test_set.sample(1, random_state=seed)
            test_set = test_set[test_set['put_id'] != selected_property['put_id'].values[0]]

            distances = test_set.distance(selected_property.geometry.iloc[0])
            test_set = test_set.loc[distances > distance_threshold]

        selected_property_reprojected = selected_property.to_crs(raster_crs)
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, selected_property_reprojected.geometry, crop=True)
            selected_pixels = (out_image != src.nodata).sum()

        accumulated_train_pixels += selected_pixels
        train_set = pd.concat([train_set, selected_property], ignore_index=True)

        print(f"Train Set: Selected PUT_ID: {selected_property['put_id'].values[0]}, pixels: {selected_pixels}, accumulated: {accumulated_train_pixels}")

    while accumulated_validation_pixels < target_validation_pixels:
        if test_set.empty:
            print("No more properties available for validation set selection.")
            break
        
        if accumulated_validation_pixels >= close_to_validation_threshold:
            print("Close to the validation pixel threshold; selecting final properties...")
            selected_property = test_set.sample(1, random_state=seed)
        else:
            selected_property = test_set.sample(1, random_state=seed)
            test_set = test_set[test_set['put_id'] != selected_property['put_id'].values[0]]

            distances = test_set.distance(selected_property.geometry.iloc[0])
            test_set = test_set.loc[distances > distance_threshold]

        selected_property_reprojected = selected_property.to_crs(raster_crs)
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, selected_property_reprojected.geometry, crop=True)
            selected_pixels = (out_image != src.nodata).sum()

        accumulated_validation_pixels += selected_pixels
        validation_set = pd.concat([validation_set, selected_property], ignore_index=True)

        print(f"Validation Set: Selected PUT_ID: {selected_property['put_id'].values[0]}, pixels: {selected_pixels}, accumulated: {accumulated_validation_pixels}")

    # Ensure the remaining properties in test_set are correctly assigned
    test_set = gdf.copy()
    test_set = test_set.loc[~test_set['put_id'].isin(train_set['put_id'])]
    test_set = test_set.loc[~test_set['put_id'].isin(validation_set['put_id'])]

    # Calculate the current percentage of the total raster area covered by training and validation
    train_percentage = (accumulated_train_pixels / total_pixels) * 100
    validation_percentage = (accumulated_validation_pixels / total_pixels) * 100
    test_pixels = total_pixels - accumulated_train_pixels - accumulated_validation_pixels
    test_percentage = (test_pixels / total_pixels) * 100

    print(f"Train Covered Percentage: {train_percentage}%")
    print(f"Validation Covered Percentage: {validation_percentage}%")
    print(f"Test Covered Percentage: {test_percentage}%")
    
    return train_set, validation_set, test_set
 """

In [ ]:
#train_set, validation_set, test_set = select_vector_sets(gdf, raster, seed = 10)

# train and test only

In [12]:
def select_vector_sets(gdf, raster_path, train_fraction=0.15, distance_threshold=5000, seed=None):
    # Set the seed for reproducibility
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    with rasterio.open(raster_path) as src:
        raster_crs = src.crs
        raster_data = src.read(1)
        total_pixels = (raster_data != src.nodata).sum()
        print(f"Total non-NA pixels in raster: {total_pixels}")

    if gdf.crs != 'EPSG:32721':
        raise ValueError("The GeoDataFrame should be in EPSG:32721.")

    # Initialize GeoDataFrames for train and test sets
    train_set = gpd.GeoDataFrame(columns=gdf.columns)
    test_set = gdf.copy()

    # Calculate the target pixel count for the training set
    target_train_pixels = train_fraction * total_pixels
    accumulated_train_pixels = 0
    close_to_train_threshold = target_train_pixels * 0.99

    print(f"Target pixel count for training set: {target_train_pixels}, Seed: {seed}")

    while accumulated_train_pixels < target_train_pixels:
        if test_set.empty:
            print("No more properties available for train set selection.")
            break

        if accumulated_train_pixels >= close_to_train_threshold:
            print("Close to the train pixel threshold; selecting final properties...")
            selected_property = test_set.sample(1, random_state=seed)
        else:
            selected_property = test_set.sample(1, random_state=seed)
            test_set = test_set[test_set['put_id'] != selected_property['put_id'].values[0]]

            distances = test_set.distance(selected_property.geometry.iloc[0])
            test_set = test_set.loc[distances > distance_threshold]

        selected_property_reprojected = selected_property.to_crs(raster_crs)
        with rasterio.open(raster_path) as src:
            out_image, out_transform = mask(src, selected_property_reprojected.geometry, crop=True)
            selected_pixels = (out_image != src.nodata).sum()

        accumulated_train_pixels += selected_pixels
        train_set = pd.concat([train_set, selected_property], ignore_index=True)

        print(f"Train Set: Selected PUT_ID: {selected_property['put_id'].values[0]}, pixels: {selected_pixels}, accumulated: {accumulated_train_pixels}")

    # Ensure the remaining properties in test_set are correctly assigned
    test_set = gdf.copy()
    test_set = test_set.loc[~test_set['put_id'].isin(train_set['put_id'])]

    # Calculate the current percentage of the total raster area covered by the training set
    train_percentage = (accumulated_train_pixels / total_pixels) * 100
    test_pixels = total_pixels - accumulated_train_pixels
    test_percentage = (test_pixels / total_pixels) * 100

    print(f"Train Covered Percentage: {train_percentage}%")
    print(f"Test Covered Percentage: {test_percentage}%")
    
    return train_set, test_set


In [14]:
train_set, test_set = select_vector_sets(gdf, raster, seed = 10)

Total non-NA pixels in raster: 84378079
Target pixel count for training set: 12656711.85, Seed: 10
Train Set: Selected PUT_ID: PUT1860, pixels: 21226, accumulated: 21226
Train Set: Selected PUT_ID: PUT0410, pixels: 3231, accumulated: 24457
Train Set: Selected PUT_ID: PUT1714, pixels: 34504, accumulated: 58961
Train Set: Selected PUT_ID: PUT1553, pixels: 189808, accumulated: 248769
Train Set: Selected PUT_ID: PUT1947, pixels: 44579, accumulated: 293348
Train Set: Selected PUT_ID: PUT1944, pixels: 4530, accumulated: 297878
Train Set: Selected PUT_ID: PUT1978, pixels: 29741, accumulated: 327619
Train Set: Selected PUT_ID: PUT0147, pixels: 7463, accumulated: 335082
Train Set: Selected PUT_ID: PUT1893, pixels: 55279, accumulated: 390361
Train Set: Selected PUT_ID: PUT0788, pixels: 6994, accumulated: 397355
Train Set: Selected PUT_ID: PUT0748, pixels: 94347, accumulated: 491702
Train Set: Selected PUT_ID: PUT2081, pixels: 23848, accumulated: 515550
Train Set: Selected PUT_ID: PUT0190, pixels

# with balanced categorical (creates issue with grupo raster of one extra pixel in width)

In [27]:
def select_vector_sets(gdf, categorical_raster_paths, continuous_raster_paths, train_fraction=0.15, distance_threshold=5000, seed=None):
    # Set the seed for reproducibility
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    # Initialize GeoDataFrames for train and test sets
    train_set = gpd.GeoDataFrame(columns=gdf.columns)
    test_set = gdf.copy()

    for raster_path in categorical_raster_paths:
        with rasterio.open(raster_path) as src:
            raster_crs = src.crs
            raster_data = src.read(1)
            total_pixels = (raster_data != src.nodata).sum()
            print(f"Processing categorical raster '{raster_path}' with {total_pixels} non-NA pixels")

        # Ensure CRS match
        if gdf.crs != 'EPSG:32721':
            raise ValueError("The GeoDataFrame should be in EPSG:32721.")

        # Calculate the target pixel count for the training set
        target_train_pixels = train_fraction * total_pixels
        accumulated_train_pixels = 0
        close_to_train_threshold = target_train_pixels * 0.99  # Define the threshold for final selection

        # Categorical raster logic
        unique_categories = np.unique(raster_data[raster_data != src.nodata])
        num_categories = len(unique_categories)

        # Track the number of pixels for each category in the training set
        category_representation_train = {category: 0 for category in unique_categories}

        # Selection loop for the training set based on categorical rasters
        distance_constraint_applied = True  # Track if distance constraint is applied

        while accumulated_train_pixels < target_train_pixels:
            if test_set.empty:
                print("No more properties available for train set selection.")
                break

            # Check if we're near the pixel threshold and begin relaxing constraints if necessary
            if accumulated_train_pixels >= close_to_train_threshold:
                print("Close to the train pixel threshold; selecting final properties...")
                selected_property = test_set.sample(1, random_state=seed)
            else:
                selected_property = test_set.sample(1, random_state=seed)
                test_set = test_set[test_set['put_id'] != selected_property['put_id'].values[0]]

                # Calculate distances and apply distance threshold
                if distance_constraint_applied:
                    distances = test_set.distance(selected_property.geometry.iloc[0])
                    test_set = test_set.loc[distances > distance_threshold]

                # If after applying distance threshold we are running out of properties, drop the distance constraint
                if test_set.empty or len(test_set) < num_categories:
                    print("Insufficient properties after applying distance threshold, relaxing constraint...")
                    distance_constraint_applied = False  # Allow selection of closer properties
                    test_set = gdf.copy()  # Reset the test set to allow re-selection without distance filtering

            selected_property_reprojected = selected_property.to_crs(raster_crs)

            with rasterio.open(raster_path) as src:
                out_image, out_transform = mask(src, selected_property_reprojected.geometry, crop=True)
                selected_pixels = (out_image != src.nodata).sum()

                # Check categories in the selected polygon and ensure representation
                categories_in_property = np.unique(out_image[out_image != src.nodata])

                for category in categories_in_property:
                    accumulated_train_pixels += selected_pixels
                    category_representation_train[category] += selected_pixels
                    train_set = pd.concat([train_set, selected_property], ignore_index=True)
                    print(f"Train Set: Selected PUT_ID: {selected_property['put_id'].values[0]}, category: {category}, pixels: {selected_pixels}, accumulated: {accumulated_train_pixels}")
                    break  # Exit after selecting one property

    # After categorical rasters, proceed to continuous rasters
    for raster_path in continuous_raster_paths:
        with rasterio.open(raster_path) as src:
            raster_crs = src.crs
            raster_data = src.read(1)
            total_pixels = (raster_data != src.nodata).sum()
            print(f"Processing continuous raster '{raster_path}' with {total_pixels} non-NA pixels")

        # Selection logic for continuous rasters (no category constraints)
        while accumulated_train_pixels < target_train_pixels:
            if test_set.empty:
                print("No more properties available for train set selection.")
                break

            if accumulated_train_pixels >= close_to_train_threshold:
                print("Close to the train pixel threshold; selecting final properties...")
                selected_property = test_set.sample(1, random_state=seed)
                print(f"Selected property: {selected_property['put_id'].values[0]}")
            else:
                selected_property = test_set.sample(1, random_state=seed)
                test_set = test_set[test_set['put_id'] != selected_property['put_id'].values[0]]

                # Calculate distances and apply distance threshold
                if distance_constraint_applied:
                    distances = test_set.distance(selected_property.geometry.iloc[0])
                    test_set = test_set.loc[distances > distance_threshold]
                    print(f"After distance filter: {len(test_set)} properties remaining")

            selected_property_reprojected = selected_property.to_crs(raster_crs)

            with rasterio.open(raster_path) as src:
                out_image, out_transform = mask(src, selected_property_reprojected.geometry, crop=True)
                selected_pixels = (out_image != src.nodata).sum()
                print(f"Selected property pixels: {selected_pixels}")

                # Continuous raster: accumulate pixels
                accumulated_train_pixels += selected_pixels
                print(f"Accumulated train pixels: {accumulated_train_pixels}/{target_train_pixels}")

                train_set = pd.concat([train_set, selected_property], ignore_index=True)
                print(f"Train Set: Selected PUT_ID: {selected_property['put_id'].values[0]}, pixels: {selected_pixels}, accumulated: {accumulated_train_pixels}")

    # Ensure the remaining properties in test_set are correctly assigned
    test_set = gdf.copy()
    test_set = test_set.loc[~test_set['put_id'].isin(train_set['put_id'])]

    return train_set, test_set



In [28]:
categorical_rasters=[r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif",r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif"]
continuous_raster = [r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\binary_deforestation_raster.tif"]

train_set, test_set = select_vector_sets(gdf=gdf, categorical_raster_paths= categorical_rasters,continuous_raster_paths= continuous_raster, seed=42)

Processing categorical raster 'C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif' with 84245038 non-NA pixels
Train Set: Selected PUT_ID: PUT0484, category: 1, pixels: 46026, accumulated: 46026
Train Set: Selected PUT_ID: PUT0787, category: 1, pixels: 6811, accumulated: 52837
Train Set: Selected PUT_ID: PUT0936, category: 1, pixels: 48326, accumulated: 101163
Train Set: Selected PUT_ID: PUT0894, category: 1, pixels: 7384, accumulated: 108547
Train Set: Selected PUT_ID: PUT1795, category: 1, pixels: 14880, accumulated: 123427
Train Set: Selected PUT_ID: PUT1491, category: 1, pixels: 23308, accumulated: 146735
Train Set: Selected PUT_ID: PUT1698, category: 1, pixels: 107789, accumulated: 254524
Train Set: Selected PUT_ID: PUT2024, category: 3, pixels: 11319, accumulated: 265843
Train Set: Selected PUT_ID: PUT0797, category: 1, pixels: 10575, accumulated: 276418
Train Set: Selected PUT_ID: PUT1703, category: 1, pixels: 54060, accumulated: 330478
Train Se

In [29]:
train_set.to_file("train_set.gpkg", driver="GPKG")

In [30]:
test_set.to_file("test_set.gpkg", driver="GPKG")

# Process Rasters

In [ ]:
""" # Open the raster file
input_raster = r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif"

with rasterio.open(input_raster, 'r+') as src:
    # Read the raster data
    data = src.read(1)  # Assuming it's a single band raster

    # Define the soil types to be merged and the new value
    soil_types_to_merge = [15, 19, 20, 22, 25]
    new_soil_value = 100  

    # Update the specified soil type values
    for soil_type in soil_types_to_merge:
        data[data == soil_type] = new_soil_value

    # Replace zero values with -1
    data[data == 0] = -1

    # Write the modified data back to the raster
    src.write(data, 1)  # Write to the first band

    # Set the nodata value to -1
    src.nodata = -1

print("Raster file updated successfully with -1 as the nodata value.") """

In [6]:
""" validation_set = gpd.read_file(r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\validation_set.gpkg") """

In [7]:
""" def process_rasters(raster_folder, validation_set, output_folder1, output_folder2):
    # Ensure the output folder exists
    os.makedirs(output_folder1, exist_ok=True)
    os.makedirs(output_folder2, exist_ok=True)


    validation_set = validation_set.to_crs('EPSG:4326')

    # Iterate over each raster file in the folder
    for raster_file in os.listdir(raster_folder):
        if raster_file.endswith('.tif'):  # Check for TIFF files
            raster_path = os.path.join(raster_folder, raster_file)

            with rasterio.open(raster_path) as src:
                # Read the full raster data
                raster_data = src.read()
                nodata_value = src.nodata


                # Mask the raster with the validation set
                masked_data, masked_transform = mask(src, validation_set.geometry, crop=True)
                masked_meta = src.meta.copy()
                masked_meta.update({"driver": "GTiff",
                                    "height": masked_data.shape[1],
                                    "width": masked_data.shape[2],
                                    "transform": masked_transform})

                # Save the masked raster
                masked_output_path = os.path.join(output_folder2, f'val_{raster_file}')
                with rasterio.open(masked_output_path, 'w', **masked_meta) as dest:
                    dest.write(masked_data)


                '''# For the inverse mask, we need to manually set the validation set areas to nodata
                inverse_masked_data = raster_data.copy()
                for geom in validation_set.geometry:
                    filled = rasterio.features.geometry_mask([geom], transform=src.transform, invert=True, out_shape=src.shape)
                    inverse_masked_data[:, filled] = nodata_value

                # Save the inverse masked (clipped) raster
                clipped_output_path = os.path.join(output_folder2, f'val_{raster_file}')
                with rasterio.open(clipped_output_path, 'w', **src.meta) as dest:
                    dest.write(inverse_masked_data)'''


                # Create and save the inverse masked (clipped) raster
                inverse_masked_data, inverse_transform = mask(src, validation_set.geometry, invert=True)
                inverse_meta = src.meta.copy()
                inverse_meta.update({"driver": "GTiff",
                                     "height": inverse_masked_data.shape[1],
                                     "width": inverse_masked_data.shape[2],
                                     "transform": inverse_transform})
                                     
                # Save the inverse masked (clipped) raster
                clipped_output_path = os.path.join(output_folder1, f'train_{raster_file}')
                with rasterio.open(clipped_output_path, 'w', **inverse_meta) as dest:
                    dest.write(inverse_masked_data)

                

                  """

In [8]:
""" process_rasters(r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters", validation_set, r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\training", r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\validation")
 """

In [ ]:
'''def select_validation_set(gdf, distance_threshold=5000, validation_fraction=0.1):
    # Initialize an empty GeoDataFrame for the validation set
    validation_set = gpd.GeoDataFrame(columns=gdf.columns)
    # Make a copy of the original GeoDataFrame to work as the remaining set
    remaining_set = gdf.copy()
    
    # Calculate the target size of the validation set based on the specified fraction
    target_size = validation_fraction * len(gdf)

    while len(validation_set) < target_size:
        # Randomly select a property from the remaining set
        selected_property = remaining_set.sample(1)

        # Append the selected property to the validation set
        validation_set = pd.concat([validation_set, selected_property], ignore_index=True)
        
        # Calculate the distance from the selected property to all properties in the remaining set
        distances = remaining_set.distance(selected_property.geometry.squeeze())
        
        # Remove properties within the distance threshold from the remaining set
        remaining_set = remaining_set.loc[distances > distance_threshold]

        # If the remaining set is empty, break the loop to prevent infinite iterations
        if remaining_set.empty:
            break

    return validation_set, remaining_set'''


In [ ]:
#validation_set, remaining_set = select_validation_set(gdf)


In [ ]:
# Save training set to a GeoPackage file
#remaining_set.to_file("training_set.gpkg", driver="GPKG")

# Save validation set to a GeoPackage file
#validation_set.to_file("validation_set.gpkg", driver="GPKG")

In [ ]:
""" # Get validation set
validation_gdf = select_validation_set(gdf)

# Get training set by excluding validation set
train_gdf = gdf[~gdf.index.isin(validation_gdf.index)]

# Save training set to a GeoPackage file
train_gdf.to_file("training_setb.gpkg", driver="GPKG")

# Save validation set to a GeoPackage file
validation_gdf.to_file("validation_setb.gpkg", driver="GPKG") """

In [ ]:
""" print(f"Training set size: {len(train_gdf)}")
print(f"Validation set size: {len(validation_gdf)}") """

In [33]:
def process_rasters(raster_folder, train_set, test_set, output_folder_train, output_folder_test):
    # Ensure the output folders exist
    os.makedirs(output_folder_train, exist_ok=True)
    os.makedirs(output_folder_test, exist_ok=True)

    # Convert the CRS of the vector sets to match the rasters (assuming 'EPSG:4326')
    train_set = train_set.to_crs('EPSG:4326')
    test_set = test_set.to_crs('EPSG:4326')

    # Iterate over each raster file in the folder
    for raster_file in os.listdir(raster_folder):
        if raster_file.endswith('.tif'):  # Check for TIFF files
            raster_path = os.path.join(raster_folder, raster_file)

            with rasterio.open(raster_path) as src:
                # Read the full raster data
                nodata_value = src.nodata

                # Process training set
                train_masked_data, train_transform = mask(src, train_set.geometry, crop=False)
                train_meta = src.meta.copy()
                train_meta.update({"driver": "GTiff",
                                   "height": train_masked_data.shape[1],
                                   "width": train_masked_data.shape[2],
                                   "transform": train_transform})

                train_output_path = os.path.join(output_folder_train, f'train_{raster_file}')
                with rasterio.open(train_output_path, 'w', **train_meta) as dest:
                    dest.write(train_masked_data)


                # Process test set
                test_masked_data, test_transform = mask(src, test_set.geometry, crop=False)
                test_meta = src.meta.copy()
                test_meta.update({"driver": "GTiff",
                                  "height": test_masked_data.shape[1],
                                  "width": test_masked_data.shape[2],
                                  "transform": test_transform})

                test_output_path = os.path.join(output_folder_test, f'test_{raster_file}')
                with rasterio.open(test_output_path, 'w', **test_meta) as dest:
                    dest.write(test_masked_data)

    print("Raster processing complete for all sets.")

In [34]:
process_rasters(
    raster_folder=r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters",
    train_set=train_set,
    test_set=test_set,
    output_folder_train=r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\training",
    output_folder_test=r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\test"
)

Raster processing complete for all sets.


In [ ]:
""" 
def create_chips(raster_path, chip_size, output_dir):
    """
"""     Create chips from a raster.

    :param raster_path: Path to the input raster file.
    :param chip_size: Size of the square chip (number of pixels).
    :param output_dir: Directory where the chips will be saved. """
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(raster_path) as src:
        width, height = src.width, src.height
        transform = src.transform

        # Calculate the number of chips in x and y directions
        x_chips = width // chip_size
        y_chips = height // chip_size

        for x in range(x_chips):
            for y in range(y_chips):
                # Calculate the window position
                window = rasterio.windows.Window(x * chip_size, y * chip_size, chip_size, chip_size)
                chip = src.read(window=window)

                # Update the transformation for the chip
                new_transform = rasterio.windows.transform(window, transform)

                # Define the output path for the chip
                chip_path = os.path.join(output_dir, f'chip_{x}_{y}.tif')

                # Save the chip
                with rasterio.open(
                    chip_path,
                    'w',
                    driver='GTiff',
                    height=chip_size,
                    width=chip_size,
                    count=src.count,
                    dtype=chip.dtype,
                    crs=src.crs,
                    transform=new_transform,
                ) as chip_file:
                    chip_file.write(chip) """



In [ ]:
""" # Example usage
raster_file = r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\binary_deforestation_raster.tif"
chip_size = 256  # Define the desired chip size (256x256 pixels)
output_directory = r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\chips"
create_chips(raster_file, chip_size, output_directory) """


# debug

In [19]:
categorical_rasters=[r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif",r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif"]
continuous_raster = [r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\binary_deforestation_raster.tif"]

for raster_path in categorical_rasters + continuous_raster:
    with rasterio.open(raster_path) as src:
        print(f"Raster {raster_path} bounds: {src.bounds}")
        print(f"Raster {raster_path} transform: {src.transform}")


Raster C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif bounds: BoundingBox(left=-62.64186038139295, bottom=-25.354320073574613, right=-57.149560721064006, top=-19.287457970745013)
Raster C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif transform: | 0.00, 0.00,-62.64|
| 0.00,-0.00,-19.29|
| 0.00, 0.00, 1.00|
Raster C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif bounds: BoundingBox(left=-62.64186038139295, bottom=-25.354320073574613, right=-57.14929123970096, top=-19.287457970745013)
Raster C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif transform: | 0.00, 0.00,-62.64|
| 0.00,-0.00,-19.29|
| 0.00, 0.00, 1.00|
Raster C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\binary_deforestation_raster.tif bounds: BoundingBox(left=-62.64186038139295, bottom=-25.354320073574613, right=-57.14929123970096, top=-19.287457970745013)
Raster C

In [20]:
for raster_path in categorical_rasters + continuous_raster:
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, train_set.geometry, crop=True)
        print(f"Raster {raster_path} original size: {src.shape}")
        print(f"Raster {raster_path} masked size: {out_image.shape}")


ValueError: Input shapes do not overlap raster.

In [21]:
for raster_path in [r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif",
                    r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif"]:
    with rasterio.open(raster_path) as src:
        train_masked_data, train_transform = mask(src, train_set.geometry, crop=True)
        print(f"Raster {raster_path}: Masked shape {train_masked_data.shape}")


ValueError: Input shapes do not overlap raster.

In [22]:
# Check CRS of raster
with rasterio.open(r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif") as src:
    raster_crs = src.crs
    print(f"Raster CRS: {raster_crs}")

# Check CRS of train_set
print(f"Train set CRS: {train_set.crs}")


Raster CRS: EPSG:4326
Train set CRS: EPSG:32721


In [24]:
# Reproject train_set to match raster CRS
train_set = train_set.to_crs(raster_crs)

# Check extent of train_set and compare to raster bounds
print(f"Train set bounds: {train_set.total_bounds}")
with rasterio.open(r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif") as src:
    print(f"Raster bounds: {src.bounds}")


Train set bounds: [-62.48935618 -24.9345956  -57.25212202 -19.37767239]
Raster bounds: BoundingBox(left=-62.64186038139295, bottom=-25.354320073574613, right=-57.149560721064006, top=-19.287457970745013)


In [25]:
# Validate geometries in train_set
train_set["is_valid"] = train_set.is_valid
print(train_set[~train_set["is_valid"]])  # Print invalid geometries, if any


Empty GeoDataFrame
Columns: [anho_capa, put_id, fecha_res, grupo, geometry, is_valid]
Index: []


In [26]:
for raster_path in [r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif",
                    r"C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif"]:
    with rasterio.open(raster_path) as src:
        train_masked_data, train_transform = mask(src, train_set.geometry, crop=False)  # Note crop=False
        print(f"Raster {raster_path}: Masked shape {train_masked_data.shape}")


Raster C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\grupo_masked.tif: Masked shape (1, 22512, 20381)
Raster C:\Users\bsf31\Documents\post-meds\data\policy-data\ml_data\rasters\soil_masked.tif: Masked shape (1, 22512, 20381)
